In [1]:
import ray
from ray import serve
import random

import requests

In [2]:
serve.start()

2022-04-12 18:56:04,726	INFO services.py:1460 -- View the Ray dashboard at http://127.0.0.1:8266
(ServeController pid=71592) 2022-04-12 18:56:08,436	INFO checkpoint_path.py:15 -- Using RayInternalKVStore for controller checkpoint and recovery.
(ServeController pid=71592) 2022-04-12 18:56:08,544	INFO http_state.py:106 -- Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:ziytnj:SERVE_PROXY_ACTOR-node:127.0.0.1-0' on node 'node:127.0.0.1-0' listening on '127.0.0.1:8000'
2022-04-12 18:56:09,786	INFO api.py:797 -- Started Serve instance in namespace 'serve'.


### Create a Python function deployment 

In [3]:
@serve.deployment
def my_ray_serve_function(request):
    args = request.query_params["data"]
    result = random.random() * float(args)
    return f"Result: {result:.3f}" 

In [4]:
my_ray_serve_function.deploy()

2022-04-12 18:56:09,807	INFO api.py:618 -- Updating deployment 'my_ray_serve_function'. component=serve deployment=my_ray_serve_function
(HTTPProxyActor pid=71609) INFO:     Started server process [71609]
(ServeController pid=71592) 2022-04-12 18:56:09,816	INFO deployment_state.py:1210 -- Adding 1 replicas to deployment 'my_ray_serve_function'. component=serve deployment=my_ray_serve_function
2022-04-12 18:56:11,815	INFO api.py:633 -- Deployment 'my_ray_serve_function' is ready at `http://127.0.0.1:8000/my_ray_serve_function`. component=serve deployment=my_ray_serve_function


### Send Request to the function 

In [5]:
for i in range(5):
    response = requests.get(f"http://127.0.0.1:8000/my_ray_serve_function?data={i}").text
    print(f'{i:2d}: {response:}')

 0: Result: 0.000
 1: Result: 0.908
 2: Result: 0.415
 3: Result: 1.428
 4: Result: 0.339


### Create a stateful class

In [17]:
from random import random
import starlette
from starlette.requests import Request

@serve.deployment
class Model:
    
    def __init__(self, threshold):
        self.threshold = threshold
        
    def __call__(self, starlette_request):
        if isinstance(starlette_request, starlette.requests.Request):
            data = starlette_request.query_params['data']
        else:
            # Request came via a ServerHandle API method call.
            data = starlette_request
        prediction = random() + float(data) if float(data) > self.threshold else float(data)
        return {"prediction": prediction}

In [18]:
Model.deploy(0.5)

2022-04-12 19:02:07,168	INFO api.py:618 -- Updating deployment 'Model'. component=serve deployment=Model
(ServeController pid=71592) 2022-04-12 19:02:07,272	INFO deployment_state.py:1167 -- Stopping 1 replicas of deployment 'Model' with outdated versions. component=serve deployment=Model
(ServeController pid=71592) 2022-04-12 19:02:09,424	INFO deployment_state.py:1210 -- Adding 1 replicas to deployment 'Model'. component=serve deployment=Model
2022-04-12 19:02:11,195	INFO api.py:633 -- Deployment 'Model' is ready at `http://127.0.0.1:8000/Model`. component=serve deployment=Model


### Send requests to the model

In [19]:
url = f"http://127.0.0.1:8000/Model"
for i in range(5):
    print(f"prediction  : {requests.get(url, params={'data': random()}).text}")

prediction  : {
  "prediction": 1.5609795907065314
}
prediction  : {
  "prediction": 0.25009238923849064
}
prediction  : {
  "prediction": 0.3763580629780887
}
prediction  : {
  "prediction": 0.30675872742057997
}
prediction  : {
  "prediction": 0.04218724913945959
}
